In [31]:
# =============================================
#
# load cifar-10-small and project down
#
 
import numpy as np
    
def unpickle( file ):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict
 
data = unpickle( 'cifar-10-batches-py/data_batch_1' )
 
features = data['data']
labels = data['labels']
labels = np.atleast_2d( labels ).T
 
N = 1000
D = 10
 
# only keep N items
features = features[ 0:N, : ] 
labels = labels[ 0:N, : ]
 
# project down into a D-dimensional space
features = np.dot( features, np.random.randn( 3072, D) )
 
# whiten our data - zero mean and unit standard deviation
features = (features - np.mean(features, axis=0)) / np.std(features, axis=0)

print features.shape
print labels.shape

W = np.random.randn(1,features.shape[1])


NUM_EPOCHS = 1000
step_size = 0.1

def loss_function(W):
    e =W
    
    

def numerical_gradient(loss_function,W):
    loss_return = loss_function()
    epsilon = 0.0001
    for i in range(1000-1):
        print i
        
    return loss_return, l 
        

for i in range(NUM_EPOCHS):
    
    scores = W.dot(features.T)
    scores -= np.max(scores, axis=0)
    scores = np.exp(scores)
    print scores.shape
    scores = scores / np.sum(scores, axis=0)
    print scores.shape
    loss = -np.log(scores)
    break

        
    
#     loss_function_value, grad = numerical_gradient( loss_function, W )
#     W = W - step_size * grad
    

(1000, 10)
(1000, 1)
(1, 1000)
(1, 1000)
